# 1. Instargram Crawling

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import time

In [6]:
# 인스타 접속
ser = Service('../chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = ser)
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(3)

In [7]:
# 검색 url 생성 함수
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/{}/'.format(word)
    return url

In [32]:
# 검색
word = '흑돼지'
url = insta_searching(word)
driver.get(url)
time.sleep(5)

In [153]:
# 검색 페이지 내 포스팅 클릭
from selenium.webdriver.common.by import By

def select_first(driver):
    #first = driver.find_element_by_css_selector('div._9AhHO')
    first = driver.find_element(By.CSS_SELECTOR, 'div._9AhH0')
    first.click()
    time.sleep(3)

select_first(driver)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [108]:
# 포스팅 내용
import unicodedata
contents = soup.select('div.C4VMK > span')[0].text
contents = unicodedata.normalize('NFC', contents)
contents

'[공덕] 솥뚜껑에 구워먹는 흑돼지 오겹살 맛집 #월화식당❤️#한국거주일본인이소개하는맛집 고기가 땡겨 월화식당에 친구들이랑 방문☺️집과 회사랑 가까워 요즘 자주 방문하는 곳이다!✨ 이날은 웨이팅이 있어 조금 대기했다가 들어갔다☺️먼저 오겹살을 주문‼️솥뚜껑에 올려서 구워 먹는 오겹살은 정말 얌암🐷미나리도 처음에 같이 나오는것도 너무 좋다❣️이날은 먹다가 차돌이랑 관자 & 묵은지 세트도 주문☺️차돌이랑 관자, 묵은지의 조합이 너무 좋았다. 묵은지가 너무 맛있는 것도 한표했다❣️그러고 치즈도 먹고 싶어서 구워 먹는 치즈도 사이드로 주문🧀일반도 있는데 슬러시소주도 있어 보통 나는 가면 슬러시로 주문!🧊🍾🐧얼음소주로 먹으면 자꾸 과음을 하게 되는데,,, oh well...+_+마지막에 김치말이국수는 주문 필수라고 이전 포스팅에서도 언급했죠!!✨ 김치말이국수와 볶음밥으로 마무리하고 배터지게 먹고 나왔다(*^ω^*)월화식당☆Tel.: 02-701-7592☆주소: 서울 마포구 도화길 29☆평일 15:00~22:00 / 주말 11:30~22:00 (거리두기 기간 동안은 21:00)☆흑돼지 오겹살 16,000원, 관자 & 묵은지세트 10,000원, 투뿔 한우 손차돌 25,000원, 구워먹는 치즈 10,000원, 김치말이국수 6,000원, 볶음밥 4,000원☆주차: 도화동주민자치센터 1시간 무료권 제공#월화식당 #오겹살 #오셥살맛집 #공덕 #공덕맛집 #아야카세 #아야카세공덕 #아야카세마포 #마포역맛집 #마포맛집 #신상맛집 #신상 #고기 #고기집 #회식장소추천 #흑돼지 #삼겹살 #도화동맛집 #도화동 #솥뚜껑삼겹살 #ソウル #ソウル旅行 #サムギョプサル #韓国 #일요일도영업 #아야카세일요일 #슬러시소주 #매일영업'

In [109]:
# 태그
import re
#tags = re.findall(r'#[^\s#,\\]+', contents)
tags = re.findall(r'#\w+', contents)
tags

['#월화식당',
 '#한국거주일본인이소개하는맛집',
 '#월화식당',
 '#오겹살',
 '#오셥살맛집',
 '#공덕',
 '#공덕맛집',
 '#아야카세',
 '#아야카세공덕',
 '#아야카세마포',
 '#마포역맛집',
 '#마포맛집',
 '#신상맛집',
 '#신상',
 '#고기',
 '#고기집',
 '#회식장소추천',
 '#흑돼지',
 '#삼겹살',
 '#도화동맛집',
 '#도화동',
 '#솥뚜껑삼겹살',
 '#ソウル',
 '#ソウル旅行',
 '#サムギョプサル',
 '#韓国',
 '#일요일도영업',
 '#아야카세일요일',
 '#슬러시소주',
 '#매일영업']

In [111]:
# 일자
date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
date

'2021-12-27'

In [128]:
# 장소
location = soup.select('a.O4GlU')[0].text
location

'월화식당'

In [122]:
# 좋아요 수
like = soup.select('div.Nm9Fw > a.zV_Nj')[0].text
like

'여러 명'

In [130]:
# 함수화

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try :
        contents = soup.select('div.C4VMK > span')[0].text
        contents = unicodedata.normalize('NFC', contents)
    except:
        contents = " "
    
    tags = re.findall(r'#\w+', contents)
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    
    try:
        like = soup.select('div.Nm9Fw > a.zV_Nj')[0].text
    except:
        like = 0
        
    try:
        place = soup.select('a.O4GlU')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place=''
        
    data = [contents, date, like, place, tags]
    return data

get_content(driver)

['[공덕] 솥뚜껑에 구워먹는 흑돼지 오겹살 맛집 #월화식당❤️#한국거주일본인이소개하는맛집 고기가 땡겨 월화식당에 친구들이랑 방문☺️집과 회사랑 가까워 요즘 자주 방문하는 곳이다!✨ 이날은 웨이팅이 있어 조금 대기했다가 들어갔다☺️먼저 오겹살을 주문‼️솥뚜껑에 올려서 구워 먹는 오겹살은 정말 얌암🐷미나리도 처음에 같이 나오는것도 너무 좋다❣️이날은 먹다가 차돌이랑 관자 & 묵은지 세트도 주문☺️차돌이랑 관자, 묵은지의 조합이 너무 좋았다. 묵은지가 너무 맛있는 것도 한표했다❣️그러고 치즈도 먹고 싶어서 구워 먹는 치즈도 사이드로 주문🧀일반도 있는데 슬러시소주도 있어 보통 나는 가면 슬러시로 주문!🧊🍾🐧얼음소주로 먹으면 자꾸 과음을 하게 되는데,,, oh well...+_+마지막에 김치말이국수는 주문 필수라고 이전 포스팅에서도 언급했죠!!✨ 김치말이국수와 볶음밥으로 마무리하고 배터지게 먹고 나왔다(*^ω^*)월화식당☆Tel.: 02-701-7592☆주소: 서울 마포구 도화길 29☆평일 15:00~22:00 / 주말 11:30~22:00 (거리두기 기간 동안은 21:00)☆흑돼지 오겹살 16,000원, 관자 & 묵은지세트 10,000원, 투뿔 한우 손차돌 25,000원, 구워먹는 치즈 10,000원, 김치말이국수 6,000원, 볶음밥 4,000원☆주차: 도화동주민자치센터 1시간 무료권 제공#월화식당 #오겹살 #오셥살맛집 #공덕 #공덕맛집 #아야카세 #아야카세공덕 #아야카세마포 #마포역맛집 #마포맛집 #신상맛집 #신상 #고기 #고기집 #회식장소추천 #흑돼지 #삼겹살 #도화동맛집 #도화동 #솥뚜껑삼겹살 #ソウル #ソウル旅行 #サムギョプサル #韓国 #일요일도영업 #아야카세일요일 #슬러시소주 #매일영업',
 '2021-12-27',
 '여러 명',
 '월화식당',
 ['#월화식당',
  '#한국거주일본인이소개하는맛집',
  '#월화식당',
  '#오겹살',
  '#오셥살맛집',
  '#공덕',
  '#공덕맛집',
  '#아야카세',
  '#아야카세공덕',
  '#아야카세

In [150]:
# 페이지 넘기기
_ = driver.find_element(By.CSS_SELECTOR, 'div.l8mY4.feth3 > button.wpO6b')
_.click()

In [151]:
# 페이지 넘기기 함수화
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, 'div.l8mY4.feth3 > button.wpO6b')
    right.click()
    time.sleep(3)

move_next(driver)

# 2. 함수화 및 결과 저장

In [158]:
## 전체 함수화

ser = Service('../chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = ser)
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(3)

def insta_crawling(word, n):
    url = insta_searching(word)
    driver.get(url)
    time.sleep(5)
    
    select_first(driver)
    time.sleep(5)
    
    target = n
    result = []
    
    for i in range(target):
        try:
            data = get_content(driver)
            result.append(data)
            move_next(driver)
        except:
            time.sleep(2)
            move_next(driver)
            
    return result

In [159]:
result_1 = insta_crawling('제주도맛집', 20)

In [161]:
len(result_1)
result_1

[['(광고)밥묵자~! 김대희가 밥두그릇 순삭했다는 그 맛집!"색달식당"\'갈치가 이렇게 큰건 처음 봐요!!\'\'와 비주얼이 대박인데요??\'\'말이 필요없는 맛!\'\'가족들이랑 꼭 다시 또 올게요!!진짜!!!\'김대희가 감탄하고 갔다는 갈치조림 맛집!크기에 놀라고~ 맛에 놀라고~정신차려보니 밥 두그릇은 기본으로 순삭~살살 녹는 부드러운 갈치살에 추위까지 살살 녹는다녹아<색달식당>*제주 서귀포시 색달중앙로 23*10:00 - 21:30 *라스트오더 20:00*브레이크타임 15:00-16:00 *064-738-1741#제주도맛집#제주갈치조림#마노르블랑#서귀포호텔#제주도여행',
  '2022-01-16',
  '여러 명',
  '',
  ['#제주도맛집', '#제주갈치조림', '#마노르블랑', '#서귀포호텔', '#제주도여행']],
 ['(광고)인스타 광고 전문업체 플랜비컴퍼니입니다.현재 수백개의 업체 인스타 광고를 통한 매출향상!매장홍보, 오픈, 신메뉴 출시 등하루 1~5만 저렴한 가격으로 인스타 인기게시물로 홍보하세요.*아무리 맛있고, 좋은 제품, 이쁜카페가 있어도, 노출이되지 않고 사람들에게 알려지지 않으면 소용없습니다. 노출이 가장중요합니다.상담문의 010 5901 1470#연남동맛집 #제주도맛집 #중문맛집 #제주공항맛집 #제주공항근처맛집 #서귀포맛집 #연남동카페 #홍대맛집 #홍대카페 #경주맛집 #서면맛집 #전포동맛집 #부산맛집 #강남맛집 #인천맛집 #대구맛집 #동성로맛집 #이태원맛집 #포항맛집 #여수맛집 #속초맛집 #강릉맛집 #홍대입구역맛집 #제주흑돼지맛집 #부평맛집 #부천맛집 #기장맛집',
  '2022-01-15',
  '여러 명',
  '',
  ['#연남동맛집',
   '#제주도맛집',
   '#중문맛집',
   '#제주공항맛집',
   '#제주공항근처맛집',
   '#서귀포맛집',
   '#연남동카페',
   '#홍대맛집',
   '#홍대카페',
   '#경주맛집',
   '#서면맛집',
   '#전포동맛집',
   '#부산맛집',
   '#

In [164]:
result_df = pd.DataFrame(result_1)
result_df.columns = ['content', 'date', 'like', 'place', 'tags']
result_df.head()

,content,date,like,place,tags
0,"(광고)밥묵자~! 김대희가 밥두그릇 순삭했다는 그 맛집!""색달식당""'갈치가 이렇게 ...",2022-01-16,여러 명,,"[#제주도맛집, #제주갈치조림, #마노르블랑, #서귀포호텔, #제주도여행]"
1,(광고)인스타 광고 전문업체 플랜비컴퍼니입니다.현재 수백개의 업체 인스타 광고를 통...,2022-01-15,여러 명,,"[#연남동맛집, #제주도맛집, #중문맛집, #제주공항맛집, #제주공항근처맛집, #서..."
2,제주 산방산 가기전 먹방투어출출할 때 유명한 #천지연숯화로구이 에서!!흑돼지오겹살 ...,2022-01-17,여러 명,,"[#천지연숯화로구이, #천지연숯화로구이, #산방산맛집, #송악산맛집, #산방산흑돼지..."
3,".❤애월 ""사이카레""❤오션뷰 바라보면서 혼밥카레도🌅너~무 분위기있다ㅠㅠ 카레도 왠지...",2022-01-14,여러 명,,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
4,<광고>#제주분식 https://jejubunsig.modoo.at/ 📌제주시 귀아...,2022-01-15,여러 명,,"[#제주분식, #제주분식, #모당모당, #돔베라면돔베김밥, #제주맛집, #제주도맛집..."


# 3. 중복제거 후 저장

In [169]:
jeju_insta_df= pd.DataFrame()
f_list = ['1_crawling_jejudoGwanGwang.xlsx',
             '1_crawling_jejudoMatJip.xlsx',
             '1_crawling_jejuMatJip.xlsx',
             '1_crawling_jejuYeoHang.xlsx',
             '1_crawling_raw.xlsx',]

for fname in f_list:
    fpath = './files/' + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)
    
    

In [173]:
print(jeju_insta_df.shape)
jeju_insta_df.head()

(21581, 5)


,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [174]:
jeju_insta_df.drop_duplicates(subset=['content'], inplace=True)
jeju_insta_df.shape

(8369, 5)

In [175]:
jeju_insta_df.to_excel('./files/1_crawling_raw.xlsx', index=False)